# Atividade: Resolver Titanic usando Keras

Crie um modelo em Keras para resolver o problema Titanic no Kaggle.

Use pipelines com scikit-learn.

Sua nota será seu score seguindo a fórmula: (score-0,65)*150, limitado a 30 pontos.

In [1]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
X = train[list(test.columns)]
y = train[train.columns[~train.columns.isin(test.columns)]]

In [2]:
import warnings
warnings.filterwarnings('ignore')

X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#preencher NaN da Cabin com outro valor para utiliza-lo de outra forma
X.fillna(X.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)

X.fillna("unknown", inplace=True)
test.fillna("unknown", inplace=True)

In [4]:
#extraindo mais caracteristicas
def extraiPronome(nome):
    return nome.split(',')[1].split('.')[0].strip()

def extraiSobrenome(name):
    return name.split(',')[0]

def extraiPrefixoCabine(cabine):
    if cabine == "unknown":
        return "Z" #categoria ficticia para os nao preenchidos
    return cabine[0]

#nova coluna com sobrenome
X["Pronome"] = X["Name"].apply(extraiPronome)
X["Sobrenome"] = X["Name"].apply(extraiSobrenome)
#X["Cabin"] = X["Cabin"].apply(extraiPrefixoCabine)

test["Pronome"] = test["Name"].apply(extraiPronome)
test["Sobrenome"] = test["Name"].apply(extraiSobrenome)
#test["Cabin"] = test["Cabin"].apply(extraiPrefixoCabine)

X.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Pronome,Sobrenome
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,unknown,S,Mr,Braund
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,Cumings
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,unknown,S,Miss,Heikkinen
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,Futrelle
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,unknown,S,Mr,Allen


A ideia foi extrair apenas o primeiro caracter do campo Cabin e verificar se isso ajuda na classificação. Além disso, o campo Sobrenome foi adicionado.

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin

def extraiPronome(nome):
    return nome.split(',')[1].split('.')[0].strip()

def extraiSobrenome(name):
    return name.split(',')[0]

def extraiPrefixoCabine(cabine):
    if cabine == "unknown":
        return "Z" #categoria ficticia para os nao preenchidos
    return cabine[0]

class AtributosDesejados(BaseEstimator, TransformerMixin):
    def __init__(self, excluirName=True, excluirCabin=False, excluirSobrenome=False):
        self.excluirName = excluirName
        self.excluirCabin = excluirCabin
        self.excluirSobrenome = excluirSobrenome
    def fit(self, X, y=None):
        #self.colunasIndesejadas = ['PassengerId', 'Ticket', 'Cabin']
        self.colunasIndesejadas = ['PassengerId', 'Ticket']
        if self.excluirName:
            self.colunasIndesejadas.append('Name')
        if self.excluirCabin:
            self.colunasIndesejadas.append('Cabin')
        if self.excluirSobrenome:
            self.colunasIndesejadas.append('Sobrenome')

        return self
    def transform(self, X, y=None):
        Xdrop = X.drop(self.colunasIndesejadas,axis=1)
        if 'Name' not in self.colunasIndesejadas:
            Xdrop['Name'] = Xdrop['Name'].apply(extraiPronome)
        
        if 'Cabin' not in self.colunasIndesejadas:
            Xdrop['Cabin'] = Xdrop['Cabin'].apply(extraiPrefixoCabine)
        return Xdrop

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas].to_numpy()

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas].to_numpy()

In [8]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

trataAtributos = Pipeline([
    ('unecaracteristicas', FeatureUnion([
        ('pipenum', Pipeline([
            ('atributos_numericos', AtributosNumericos()),
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])),
        ('pipecat', Pipeline([
            ('atributos_categoricos', AtributosCategoricos()),
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]))
    ])),
])

In [9]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedKFold

#classifier = RandomForestClassifier()
def TreinarPadrao(classifier):
    pipetotal = Pipeline([
        ('atributosDesejados', AtributosDesejados()),
        ('trataAtributos', trataAtributos),
        ('classificador', classifier) 
    ])

    parametros = {
        'atributosDesejados__excluirName': [True, False],
        'atributosDesejados__excluirCabin': [True],
        'atributosDesejados__excluirSobrenome': [True], #após alguns testes mantive as duas informações novas. Como demora muito, estou fizando em True
        'classificador__max_depth': [5]
    }

    modelo = GridSearchCV(pipetotal, param_grid=parametros)

    scores = cross_validate(modelo, X, y, cv=RepeatedKFold())
    scores['test_score'], np.mean(scores['test_score']), np.std(scores['test_score'])

In [10]:
#
def FitAndPrintModelToKaggle(modelo, X, y, fit=True):
    if fit:
        modelo.fit(X,y)
    
    y_pred = modelo.predict(test)
    result = test[['PassengerId']]
    result['Survived'] = y_pred
    result.to_csv('submission.csv',index=False)

## Definição do MLP com Keras


### Funções de plot

As funções abaixo são úteis para testes cuja base de dados seja plotável para as acaracterísticas que se desejar.

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.preprocessing import label_binarize

def plotDataSet(X,y):
    plt.xlabel('X1')
    plt.ylabel('X2')
    
    for k in set(y):
        #print("k=",k)
        plt.plot(X[:,0][y==k],
                 X[:,1][y==k],
                 "o",alpha=0.3)

def plotHiperplano(X,y,vetor, intercept=0):
    x0min = min(X[:,0])
    x0max = max(X[:,0])
    
    xs = np.linspace(x0min, x0max, num=2)
    #separador do hiperplano entre duas classificações pode ser 
    #encontrada conforme calculo abaixo:
    ys = (-vetor[0]/vetor[1])*xs-intercept/vetor[1]
    plt.plot(xs,ys)

def define_axes(X, margem=0.1):
    '''
    A função retorna os eixos com a margem de 0.1
    O vetor será o mínimo até o máximo de cada um; esses são os eixos em que faremos a distribuição.
    '''
    #o mínimo e máximo da característica 1
    min1 = X[:,0].min()
    max1 = X[:,0].max() 

    min2 = X[:,1].min()
    max2 = X[:,1].max() 

    #colocando margem para baixo e margem pra cima de 10%
    return [min1-margem, max1+margem, min2-margem, max2+margem] #margem idel é .1 é o mesmo que 0.1

def plotPredictions(clf, X):
    '''
    Recebe um classificador e a matriz de características e exibe um gráfico utilizando meshgrid.
    '''
    axes = define_axes(X)
    x0s = np.linspace(axes[0], axes[1], 100)
    x1s = np.linspace(axes[2], axes[3], 100)
    x0,x1 = np.meshgrid(x0s, x1s) #está fazendo a distribuição nxn, gerando duas matrizes
    X_ = np.c_[x0.ravel(), x1.ravel()] #c_ = cópia
    ypred  = clf.predict(X_).reshape(x0.shape)
    plt.contourf(x0,x1,ypred,cmap=plt.cm.brg, alpha=0.2)

A seguir, criaremos a classe MLP com Keras.

Note que o predict praticamente não mudou desde a última atividade, porém, o fit está totalmente adaptado ao uso da biblioteca do Keras.
Mantive os parâmetros básicos de configuração da rede. Achei muito útil o uso do parâmetro use_multiprocessing=True no fit do modelo. Acelerou bem o tempo de treinamento.

O passo a passo de definição da rede segue basicamente o mesmo que vimos em sala de aula:<br>
1 - Faz o yhot_encode dos doados de entrada <br>
2 - InputLayer com quantidade de neurônios conforme a quantidade de características base de dados<br>
3 - Adiciona as camadas intermediárias conforme parametrização passando a função de ativação parametrizada<br>
4 - Como precisamos construir uma rede tal que possa processar tanto bases binárias quanto multiclasse, precisamos observar que a definição da quatidade de neurônios da última camada deve ser conforme a base de dados. A função de ativação softmax pode ser usada em ambos casos, portanto foi escolhida.<br>
5 - Chama o fit<br>

In [12]:
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
from tensorflow import keras
from keras import layers
from tensorflow.keras.utils import to_categorical

class MLPKerasClassifier(BaseEstimator, ClassifierMixin):
    '''
    Rede neural para classificação utilizando Keras.
    '''
    def __init__(self, n_hidden=[5,4], activation_function='relu', epochs=1000, verbose=0, callbacks=[]):
        self.n_hidden = n_hidden
        self.activation_function = activation_function
        self.epochs = epochs
        self.verbose = verbose
        self.callbacks = callbacks
    
    def fit(self, X, y):
        #adicionar ao self no final
        self.labels,ids = np.unique(y,return_inverse=True)
        
        #tratando caso binario ou multiclasse
        yhot = keras.utils.to_categorical(ids)
        size_last_layer = yhot.shape[1]
        size_first_layer = X.shape[1]
        
        if len(self.n_hidden) == 0:
            raise Exception('Parâmetro inválido: n_hidden!')
        else:
            #inicia o modelo
            model = keras.models.Sequential()
            model.add(layers.InputLayer(input_shape=(size_first_layer,)))

            #para cada elemento da lista, adicionar uma camada densa com a quantidade de neurônios conforme valor.
            for idx in range(len(self.n_hidden)):
                value = self.n_hidden[idx]
                model.add(layers.Dense(value, activation=self.activation_function))

            #na última camada, função softmax pra ser capaz de classificar bases binárias ou multiclasse
            model.add(layers.Dense(size_last_layer,activation='softmax'))

            model.compile(optimizer="rmsprop",
                            loss='categorical_crossentropy', #precisa ser categorial_... porque a base pode ser multiclasse
                            metrics=['accuracy'])
            
            model.fit(X,yhot,epochs=self.epochs,use_multiprocessing=True, verbose=self.verbose, callbacks=self.callbacks)
            self.coef_ = model.get_weights() 
            
            #salva o modelo
            self.model = model
        

    def predict(self, X, y=None):
        a = self.model.predict(X)
        idx = np.argmax(a, axis=1)
        ypred = self.labels[idx]
        return ypred

### Otimizando parâmetros para SVM, LogisticRegression, RandomForest e KNN

Testaremos otimizar os parâmetros para estes dois classificadores e depois utilizá-lo em um stacking no final.

In [13]:
#separando parte da base de treino e teste para validação do modelo e otimização de parâmetros
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((668, 13), (223, 13), (668, 1), (223, 1))

In [14]:
#Construindo um stacking de RandomForest aplicando GridSerach para otimizar os parâmetros do classficiador e utilizar no final
from sklearn.metrics import make_scorer, mean_squared_error

def BuildTestModel(model, parameters, x_tr, y_tr):
    '''
    Recebe um modelo de classificador e os parâmetros desejados para otimizar no parâmetro. Constroi o modelo, oferece a acuracia e retorna o modelo.
    '''
    pipetotal = Pipeline([
        ('atributosDesejados', AtributosDesejados()),
        ('trataAtributos', trataAtributos),
        ('classificador', model)
    ])

    parametros = dict()
    parametros['atributosDesejados__excluirName'] = [True]
    parametros['atributosDesejados__excluirCabin'] = [False]
    parametros['atributosDesejados__excluirSobrenome'] = [False]
    
    for i in range(len(parameters)):
        parametros['classificador__' + parameters[i][0]] = parameters[i][1]

    modelo = GridSearchCV(pipetotal, param_grid=parametros)
    #modelo = pipetotal
    modelo.fit(x_tr, y_tr)

    #scores = cross_validate(modelo, x_tr, y_tr, cv=RepeatedKFold(n_repeats=10))
    scores = cross_validate(modelo, x_tr, y_tr)
    
    print("Score médio:", np.mean(scores['test_score']), "\nDesvio P.:", np.std(scores['test_score']))

    return modelo

### Treinando o 

Caso queira-se buscar melhores valores nos parâmetros da rede MLP, pode-se destacá-los para serem chamados no GridSearch conforme abaixo por meio da lista de parâmetros em parMLP.
Além disso, passamos também como opção, função de callback EarlyStopping para parar o treinamento da rede caso a acurácia caia duas vezes consecutivas.

In [18]:
parMLP = []
#parMLP.append(['C', [0.1, 1, 2]]) 
#parMLP.append(['gamma', [0.01,0.001]])
#parMLP.append(['kernel', ['rbf', 'poly', 'sigmoid']])


callbacks_list = [
    keras.callbacks.EarlyStopping(
    monitor="accuracy", #função a ser considerada no monitoramento para pararmos o treinamento
    patience=2)#,     #se aumentar a acurácia de validação duas vezes ele para

    #keras.callbacks.ModelCheckpoint(filepath="checkpoint_path.keras",
    #                            monitor="val_loss",
    #                            save_best_only=True)
]

modMLP = BuildTestModel(MLPKerasClassifier(n_hidden=[16,8], activation_function='relu', epochs=1000, verbose=1, callbacks=callbacks_list), parMLP, X_train, y_train)

#caso utilize GridSearch
modMLP.best_params_
modMLP.best_estimator_

Epoch 1/1000
17/17 [==============================] - 1s 2ms/step - loss: 0.6569 - accuracy: 0.6629
Epoch 2/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.6185 - accuracy: 0.7116
Epoch 3/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.5851 - accuracy: 0.7453
Epoch 4/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.5532 - accuracy: 0.7809
Epoch 5/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.5223 - accuracy: 0.8090
Epoch 6/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.4920 - accuracy: 0.8333
Epoch 7/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.4637 - accuracy: 0.8483
Epoch 8/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.4389 - accuracy: 0.8464
Epoch 9/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.4180 - accuracy: 0.8614
Epoch 10/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.3992 - accuracy: 0.8521

Pipeline(steps=[('atributosDesejados', AtributosDesejados()),
                ('trataAtributos',
                 Pipeline(steps=[('unecaracteristicas',
                                  FeatureUnion(transformer_list=[('pipenum',
                                                                  Pipeline(steps=[('atributos_numericos',
                                                                                   AtributosNumericos()),
                                                                                  ('imputer',
                                                                                   SimpleImputer(strategy='median')),
                                                                                  ('scaler',
                                                                                   StandardScaler())])),
                                                                 ('pipecat',
                                                                  Pipel

In [16]:
def FitPredictToCsv(model, filename, x_tr, y_tr):
    model.fit(x_tr,y_tr)
    y_pred = model.predict(test)
    s = cross_validate(model, x_tr, y_tr, cv=RepeatedKFold())
    print("SCORE FINAL\nMédio:", np.mean(s['test_score']), "\nDesvio P.:", np.std(s['test_score']))

    result = test[['PassengerId']]
    result['Survived'] = y_pred
    result.to_csv(filename+'.csv',index=False)

def PredictToCsv(model, filename):   
    y_pred = model.predict(test)

    result = test[['PassengerId']]
    result['Survived'] = y_pred
    result.to_csv(filename+'.csv',index=False)

In [19]:
PredictToCsv(modMLP, "kaggle_submission_v2")